In [1]:
import os
import numpy as np
import pandas as pd
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from tqdm import tqdm

# ==== CONFIGURATION ====
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.empty_cache()
print(f"Using device: {device}")

PATCH_SIZE = 64  # As per paper
NUM_FRAMES = 10
NUM_BINS = 12

Using device: cuda


In [2]:
# ==== Y_SPEED VECTOR COMPUTATION ====
def compute_ground_truth_speed(optical_flows, num_bins=12):
    speed_vectors = []
    for flow in optical_flows:
        flow_x = flow[..., 0]
        flow_y = flow[..., 1]
        mag, ang = cv2.cartToPolar(flow_x.astype(np.float32), flow_y.astype(np.float32))
        hist = np.zeros(num_bins)
        bin_edges = np.linspace(0, 2 * np.pi, num_bins + 1)
        for i in range(num_bins):
            bin_mask = (ang >= bin_edges[i]) & (ang < bin_edges[i + 1])
            if np.sum(bin_mask) > 0:
                hist[i] = np.mean(mag[bin_mask])
        speed_vectors.append(hist)
    return np.mean(speed_vectors, axis=0)

In [3]:
from tqdm import tqdm

# Updated with inline progress print during volume collection
def load_data_from_directory_with_debug(base_dir):
    video_volumes = []
    speed_vectors = []

    all_volume_paths = []
    print(f"🔍 Scanning base directory: {base_dir}")
    for subdir in sorted(os.listdir(base_dir)):
        sub_path = os.path.join(base_dir, subdir)
        video_path = os.path.join(sub_path, "video_volumes")
        flow_path = os.path.join(sub_path, "optical_flows")

        if not os.path.isdir(video_path) or not os.path.isdir(flow_path):
            print(f"⚠️ Skipping {subdir}: Missing video_volumes or optical_flows folder.")
            continue

        print(f"⏳ Collecting volume paths in {subdir}...")
        for vol_folder in sorted(os.listdir(video_path)):
            vol_full_path = os.path.join(video_path, vol_folder)
            flow_folder_path = os.path.join(flow_path, vol_folder)
            if os.path.isdir(vol_full_path) and os.path.isdir(flow_folder_path):
                all_volume_paths.append((vol_full_path, flow_folder_path))

    print(f"✅ Total volumes to load: {len(all_volume_paths)}")

    for vol_full_path, flow_folder in tqdm(all_volume_paths, desc="📥 Loading volumes"):
        frames = []
        for i in range(NUM_FRAMES):
            frame_path = os.path.join(vol_full_path, f"frame_{i:04d}.png")
            img = cv2.imread(frame_path)
            if img is None:
                print(f"⚠️ Skipped volume {vol_full_path}: Missing frame {i}")
                frames = []
                break
            img = cv2.resize(img, (PATCH_SIZE, PATCH_SIZE))
            frames.append(img)

        if len(frames) == NUM_FRAMES:
            video_volumes.append(np.stack(frames, axis=0))

        flows = []
        for i in range(NUM_FRAMES - 1):
            flow_path_i = os.path.join(flow_folder, f"flow_raw_{i:04d}.npy")
            if not os.path.exists(flow_path_i):
                print(f"⚠️ Missing flow file: {flow_path_i}")
                flows = []
                break
            flow = np.load(flow_path_i)
            flow = cv2.resize(flow, (PATCH_SIZE, PATCH_SIZE))
            flows.append(flow)

        if len(flows) == NUM_FRAMES - 1:
            speed_vector = compute_ground_truth_speed(flows, NUM_BINS)
            speed_vectors.append(speed_vector)

    if not video_volumes or not speed_vectors:
        raise ValueError("🚫 No valid video volumes or speed vectors were found.")

    X = np.stack(video_volumes)  # (N, 10, H, W, C)
    X = X.transpose(0, 4, 1, 2, 3)  # (N, C, T, H, W)
    Y = np.stack(speed_vectors)    # (N, 12)

    print(f"✅ Loaded {len(X)} video volumes and {len(Y)} Y_speed vectors.")
    return torch.tensor(X, dtype=torch.float32), torch.tensor(Y, dtype=torch.float32)


In [4]:
# ==== MODEL DEFINITION ====
class YSpeedModel(nn.Module):
    def __init__(self):
        super(YSpeedModel, self).__init__()
        self.conv1 = nn.Conv3d(3, 32, kernel_size=(5,5,5), padding=2)
        self.bn1 = nn.BatchNorm3d(32)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool3d(kernel_size=(1,2,2), stride=(1,2,2))

        self.conv2 = nn.Conv3d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm3d(64)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool3d(kernel_size=(1,2,2), stride=(1,2,2))

        self.conv3 = nn.Conv3d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm3d(128)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool3d(kernel_size=(1,2,2), stride=(1,2,2))

        self.fc1 = nn.Linear(128 * (PATCH_SIZE//8) * (PATCH_SIZE//8) * NUM_FRAMES, 256)
        self.fc_bn = nn.BatchNorm1d(256)
        self.fc_relu = nn.ReLU()
        self.fc2 = nn.Linear(256, NUM_BINS)

    def forward(self, x, extract_features=False):
        x = self.pool1(self.relu1(self.bn1(self.conv1(x))))
        x = self.pool2(self.relu2(self.bn2(self.conv2(x))))
        x = self.pool3(self.relu3(self.bn3(self.conv3(x))))
        x = torch.flatten(x, 1)
        x = self.fc_relu(self.fc_bn(self.fc1(x)))
        return x if extract_features else self.fc2(x)

In [5]:
# ==== TRAINING FUNCTION ====
def train_model(model, train_loader, val_loader, epochs=30):
    model.train()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    criterion = nn.MSELoss()

    for epoch in tqdm(range(epochs), desc="Training YSpeed Model"):
        model.train()
        total_loss = 0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}: Loss = {total_loss / len(train_loader):.4f}")

In [6]:
    # ==== LOAD DATA ====
    base_dir = "C:\\Users\\hci\\Desktop\\data_new1"
    video_vols, speed_vecs = load_data_from_directory_with_debug(base_dir)

🔍 Scanning base directory: C:\Users\hci\Desktop\data_new1
⏳ Collecting volume paths in V1...
⏳ Collecting volume paths in V10...
⏳ Collecting volume paths in V11...
⏳ Collecting volume paths in V12...
⏳ Collecting volume paths in V13...
⏳ Collecting volume paths in V14...
⏳ Collecting volume paths in V15...
⏳ Collecting volume paths in V16...
⏳ Collecting volume paths in V17...
⏳ Collecting volume paths in V18...
⏳ Collecting volume paths in V19...
⏳ Collecting volume paths in V2...
⏳ Collecting volume paths in V20...
⏳ Collecting volume paths in V21...
⏳ Collecting volume paths in V3...
⏳ Collecting volume paths in V4...
⏳ Collecting volume paths in V5...
⏳ Collecting volume paths in V6...
⏳ Collecting volume paths in V7...
⏳ Collecting volume paths in V8...
⏳ Collecting volume paths in V9...
✅ Total volumes to load: 209848


📥 Loading volumes: 100%|████████████████████████████████████████████████████| 209848/209848 [5:24:33<00:00, 10.78it/s]


✅ Loaded 209848 video volumes and 209848 Y_speed vectors.


In [7]:
dataset = TensorDataset(video_vols, speed_vecs)
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_ds, val_ds = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=20, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=20)

In [37]:
# ==== TRAIN MODEL ====
model = YSpeedModel().to(device)
train_model(model, train_loader, val_loader, epochs=30)

Training YSpeed Model:   3%|█▉                                                       | 1/30 [10:54<5:16:14, 654.30s/it]

Epoch 1: Loss = 0.0988


Training YSpeed Model:   7%|███▊                                                     | 2/30 [21:46<5:04:47, 653.13s/it]

Epoch 2: Loss = 0.0695


Training YSpeed Model:  10%|█████▋                                                   | 3/30 [32:35<4:53:07, 651.39s/it]

...

Epoch 28: Loss = 0.0122


Training YSpeed Model:  97%|██████████████████████████████████████████████████████▏ | 29/30 [5:15:13<10:52, 652.86s/it]

Epoch 29: Loss = 0.0120


Training YSpeed Model: 100%|████████████████████████████████████████████████████████| 30/30 [5:26:04<00:00, 652.15s/it]

Epoch 30: Loss = 0.0116


In [38]:
torch.save(model.state_dict(), "y_speed_model.pth")
print("✅ YSpeed model training completed and saved.")

✅ YSpeed model training completed and saved.


In [8]:
model_speed = YSpeedModel().to(device)
model_speed.load_state_dict(torch.load("y_speed_model.pth"))

C:\Users\hci\AppData\Local\Temp\ipykernel_13088\3350178538.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_speed.load_state_dict(torch.load("y_speed_model.pth"))


<All keys matched successfully>

In [9]:
# ✅ Extract feature vectors
def extract_feature_vectors(model, dataloader, save_name):
    model.eval()
    features = []

    with torch.no_grad():
        for inputs, _ in dataloader:
            inputs = inputs.to(device)
            feature_vecs = model(inputs, extract_features=True)
            features.append(feature_vecs.cpu().numpy())

            
            

    features = np.vstack(features)
    np.save(f"{save_name}.npy", features)
    pd.DataFrame(features).to_csv(f"{save_name}.csv", index=False)

    print(f"✅ Feature vectors saved: {save_name}")

In [10]:
# ✅ **Extract Feature Vectors**
features = extract_feature_vectors(model_speed, train_loader, "y_speed_feature_updated_test.csv")
print("✅ Feature vectors extracted and saved successfully!")

✅ Feature vectors saved: y_speed_feature_updated_test.csv
✅ Feature vectors extracted and saved successfully!
